# **Example of wrappers.rendering**
ソースコードは以下から参照されたい  
[Source code for gymnasium.wrappers.rendering](https://gymnasium.farama.org/main/_modules/gymnasium/wrappers/rendering/#HumanRendering)

In [ ]:
if str(get_ipython()).startswith('<google.colab.'):
    !pip install gymnasium

## <font color= "#ff2200">**RenderCollectionラッパーの使い方**</font>

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RenderCollection

# 環境を作成し、RenderCollectionラッパーを適用
env = gym.make("CartPole-v1", render_mode="rgb_array")
env = RenderCollection(env)

# 環境をリセット
observation, info = env.reset(seed=42)

# 200ステップ実行し、フレームを収集
frames = []
for _ in range(200):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break

# 収集したフレームを取得
frames = env.render()

# フレーム数を出力
print(f"Collected {len(frames)} frames.")

env.close()

### **使い方**

RenderCollectionラッパーは、環境のレンダリングされたフレームをリストに収集するためのものです。以下のように適用します。

```python
import gymnasium as gym
from gymnasium.wrappers import RenderCollection

env = gym.make("env_id", render_mode="rgb_array")
env = RenderCollection(env)
```

レンダリングされたフレームは `env.render()` を呼び出すことで取得できます。さらに、以下のパラメータを指定できます。

- `pop_frames` (デフォルト: True) - `render()`が呼び出された後にフレームリストを空にするかどうか
- `reset_clean` (デフォルト: True) - `reset()`が呼び出された時にフレームリストを空にするかどうか



上記のコードでは、CartPole-v1環境を作成し、RenderCollectionラッパーを適用しています。その後、200ステップ実行し、レンダリングされたフレームを収集しています。

最後に `env.render()` を呼び出すと、収集したフレームのリストが返されます。デフォルトでは、この呼び出し後にフレームリストは空になります(`pop_frames=True`)。

出力例:

```
Collected 200 frames.
```

収集したフレームは、後で動画に変換したり、フレーム単位での処理に使用できます。RenderCollectionは、レンダリングされた環境の経過を記録しておく際に便利です。

## <font color= "#00ff00">**RecordVideoラッパーの使い方**</font>

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo

# 環境の作成
env = gym.make("CartPole-v1", render_mode="rgb_array")

# ビデオ記録ラッパーの適用
env = RecordVideo(env, "videos/")

# エピソードのループ
for episode in range(10):
    observation, info = env.reset()
    done = False
    while not done:
        # エージェントの行動を取得 (ここではランダムに行動)
        action = env.action_space.sample()

        # 1ステップ実行し、次の状態を取得
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

# 環境のクローズ
env.close()


### **使い方**
上記の例では、以下の手順を行っています。

1. `gymnasium`環境を作成(`CartPole-v1`の例)。`render_mode="rgb_array"`を指定して画像形式での描画を有効にします。
2. `RecordVideo`ラッパーを適用し、`"videos/"`ディレクトリにビデオを保存するよう設定します。
3. 10エピソードのループを回し、各エピソードでランダムな行動をとります。
4. 各エピソードが終了するたびに、自動的にビデオ記録が開始/停止されます。
5. 全エピソード終了後に`env.close()`で環境を閉じます。

この例では、`RecordVideo`ラッパーのデフォルト設定が使用されています。デフォルトでは、`capped_cubic_video_schedule`関数に基づいてビデオ記録のトリガーが決定されます。この関数は、エピソード番号が0、1、8、27...の時にビデオ記録を開始するように設計されています。

記録されたビデオファイルは`videos/`フォルダに保存され、ファイル名は`"rl-video-episode-{episode_id}"`という形式になります。

必要に応じて、`RecordVideo`の初期化時に以下のパラメータを指定することもできます。

- `episode_trigger`: エピソードトリガー関数を指定
- `step_trigger`: ステップトリガー関数を指定
- `video_length`: 記録するビデオの長さ (フレーム数) を指定
- `name_prefix`: ビデオファイル名のプレフィックスを変更

このように、`RecordVideo`ラッパークラスを使うことで、Gymnasium環境での学習やテストの過程をビデオとして自動的に記録し、後で振り返ることができます。


### **初期化時の設定項目**
`RecordVideo`クラスの初期化時に指定できるパラメータは以下の通りです。

```python
RecordVideo(env,
            video_folder,
            episode_trigger=None,
            step_trigger=None,
            video_length=0,
            name_prefix='rl-video',
            disable_logger=False)
```

- `env` (gymnasium.Env): ラップする環境オブジェクト
- `video_folder` (str): ビデオを保存するフォルダのパス
- `episode_trigger` (Callable[[int], bool], optional): エピソードトリガー関数。引数にエピソード番号を取り、そのエピソードでビデオ記録を開始するかどうかをブール値で返す。
- `step_trigger` (Callable[[int], bool], optional): ステップトリガー関数。引数にステップ数を取り、そのステップでビデオ記録を開始するかどうかをブール値で返す。
- `video_length` (int, optional): 記録するビデオの長さ(フレーム数)。0を指定するとエピソード全体を記録する。
- `name_prefix` (str, optional): 保存されるビデオファイル名のプレフィックス。デフォルトは'rl-video'。
- `disable_logger` (bool, optional): MoviePyのロガーを無効にするかどうか。デフォルトはFalse。

`episode_trigger`と`step_trigger`はどちらか一方しか指定できません。両方指定した場合はAssertionErrorが発生します。

また、`episode_trigger`と`step_trigger`を指定しない場合は、デフォルトで`capped_cubic_video_schedule`関数がエピソードトリガーとして使用されます。

例えば、10エピソードごとにビデオ記録を開始するようにするには、次のように`episode_trigger`を指定します:

```python
env = gym.make("CartPole-v1")
trigger = lambda episode_id: episode_id % 10 == 0
env = RecordVideo(env, "videos/", episode_trigger=trigger)
```

最大1000フレームまでのビデオを記録し、ファイル名に"run1"を含めるには:

```python
env = RecordVideo(env, "videos/", video_length=1000, name_prefix="run1")
```

このように、`RecordVideo`ラッパーの初期化時にパラメータを設定することで、ビデオ記録の動作をカスタマイズできます。

## <font color= "#0055ff">**HumanRenderingクラスの使い方**</font>

In [ ]:
import gymnasium as gym

# 環境作成時にHumanRenderingラッパーを適用
env = gym.make("CartPole-v1", render_mode="human")

# 環境をリセット
observation, info = env.reset(seed=42)

# 50ステップ実行しながら、レンダリングを行う
for _ in range(50):
    action = env.action_space.sample() # ランダムなアクションを選択
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

### 使い方

HumanRenderingラッパーは、環境の"rgb_array"レンダリングモードをウィンドウに表示するためのものです。以下の2つの方法でラッパーを適用できます。

1. **環境ラッパー適用後**

```python
import gymnasium as gym
from gymnasium.wrappers import HumanRendering

env = gym.make("env_id", render_mode="rgb_array")
env = HumanRendering(env)
```

2. **環境作成時**

```python
import gymnasium as gym

env = gym.make("env_id", render_mode="human")
```

この方法では、環境が"human"レンダリングモードをネイティブにサポートしていない場合にのみ、HumanRenderingラッパーが自動的に適用されます。

上記のコードを実行すると、LunarLanderの画面がウィンドウに表示され、環境が50ステップ進行するたびにフレームがレンダリングされます。

環境がリセットされるたびに、新しいエピソードが始まり、ウィンドウにレンダリングされた最初のフレームが表示されます。

HumanRenderingラッパーは、人間が環境を視覚化したり、動作を確認したりするのに便利です。ただし、レンダリングの [overhead](https://hldc.co.jp/blog/2022/12/15/11611/) が高いため、実際の強化学習トレーニングなどでは使用しないことをお勧めします。